### třetí lekce

2. `pandas` I.,
    - Úvod do frameworku, proč je dobré jej ovládat, kde s ním pracovat?
    - jaké jsou základní datové typy? (Dframe, series, časové řady)
    - základní orientace, indexování,

## Druhá lekce, kaggle
* reading and writting DFs,
* indexing, selecting rows & columns, conditional selection,
* functions, map and lambdas,
* groupby, multiindex, sorting values,
* data types or Dtypes, convert data types,
* missing values, fillna, replace,
* rename, merge, join, concat.
* missing data points, the reason why are the data missing, drop missing values, filling missing vals,
* scaling: changing the range, normalization: changing shape,
* parsing dates, selecting days,
* encoding: what are encodings, encode, decode, identifying and saving examples,
* inconsistency: fuzzywuzzy

## Druhá lekce, kniha
* pandas objects: DFs, Series, Index,
* indexing and selection: series, DFs,
* operating: w/ numpy (hard to implement without numpy),
* handling missing data: Nan and None in pandas,detecting, droping, filling null values,
* indexing: single index, multiple indexes(bad and good example), indexing and slicing multiindex,
* concat, append, merge, join,
* agregation: count, first, last, mean, median, min, max,
* groupBy: split, apply, combine,
* pivot table, methods similar to python methods, regex finding,slice & other miscellaneous methods,
* Time series, to_datetime, data structures of time series, date_range, period_range, timedelta_range,
* frenquencies & offsets, resampling, time shifting,
* eval and query, comparing eval and without eval,


## Druhá lekce, Honza
* manipulace, úvod, instalace,
* DataFrame,
* Series,
* Načítání dat,
* selekce dat,
* indexování, loc a iloc,
* přejmenování, přidávání, odstraňování sloupců,
* Nan, null, nahrazování,
* concat, merge, drop_duplicates, sort_values,
* unique, apply,
* kopie a práce s kopiemi,
* statistika,
* čtení dat online,

* [Spojování dat](#Spojování-dat),
    - [spojování s concat](#Spojování-s-concat),
    - [spojování s concat a arg. join](#concat-a-volitelný-argument-join),
    - [spojování s metodou append](#Spojování-pomocí-metody-append),
    - [spojování s merge](#Spojování-pomocí-merge),
    - [spojování s metodou join](#Spojení-pomocí-indexů,-join),
    - [cvičení 1]().
* [Agregace](),
    - [jednoduchá agregace](),
    - [seskupování groupby](),
    - [agregace](),
    - [filtrování](),
    - [transformace](),
    - [apply](),
    - [cvičení 2]().
* [Pivot tabulka](),
    - [úvodní motivace](),
    - [syntaxe tabulky](),
    - [doplňující možnosti](),
    - [cvičení 3]().
* [Časové řady](),
    - [úvodní motivace](),
    - [datum & čas](),
    - [četnosti](),
    - [resampling](),
    - [cvičení 4]().
* [High performance](),
    - [úvodní motivace](),
    - [eval](),
    - [query](),
    - [caveats]().

<br>

## Spojování dat

---


<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.APOjuHGvkM0wQaUx9ELKAgHaHa%26pid%3DApi&f=1&ipt=a8b2b692c40e190d4b28c164bf13e6c944702a69944035a4ef9053aa6ea1b190&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

Takovými operacemi si můžeš představit jednoduché operace jako **spojování** (*konkatenace*) **dvou a více datasetů** až po složitější joiny podobně jako u databází.

`pandas` obsahuje různé *funkce* a *metody*, které umožňují tento způsob práce.

### Spojování s `concat`

---

Funkce concat() v pandas umožňuje spojit dva nebo více:
1. sloupečků,
2. tabulek.

*Spojení* můžeš provést:
1. **horizontálně**, po sloupcích,
2. **vertikálně**, po řádcích.

Použití funkce `concat()` spočívá v předání seznamu objektů, které chceš spojit, a parametru `axis`, který určuje osu, podle které se májí objekty spojit:

#### Spojení sloupečků

In [ ]:
from pandas import concat, Series, DataFrame

In [ ]:
sloupec_1 = Series(['A', 'B', 'C'], index=[1, 2, 3])
sloupec_2 = Series(['D', 'E', 'F'], index=[4, 5, 6])

In [ ]:
spojene_sloupce = concat([sloupec_1, sloupec_2])

In [ ]:
spojene_sloupce

#### Spojení tabulek

In [ ]:
uzivatele_1 = {
    'jmeno': ['Matouš', 'Marek', 'Lukáš'],
    'vek': [25, 30, 35]
}

In [ ]:
uzivatele_2 = {
    'jmeno': ['Petr', 'Jan', 'Michal'],
    'vek': [40, 45, 50]
}

In [ ]:
df_uzivatele_1 = DataFrame(uzivatele_1)
df_uzivatele_2 = DataFrame(uzivatele_2)

In [ ]:
df_spojene = concat([df_uzivatele_1, df_uzivatele_2], axis=0)

In [ ]:
df_spojene.head()

Tady se ovšem zduplikovali hodnoty některých indexů.

Ty je potřeba opravit tímto postupem:
1. **Vytvořím nový sloupeček** pro indexy,
2. **odstraním starý sloupeček** s duplicitami.

In [ ]:
df_spojene = df_spojene.reset_index()

In [ ]:
df_spojene.head()

In [ ]:
df_spojene = df_spojene.drop("index", axis=1)

In [ ]:
df_spojene

Pro odchytávání duplicitních indexů můžeš doplnit parametr `verify_integrity=True`, případně pokud je irelevantní, ignorovat jej úplně `ignore_index=True`.

### `concat` a volitelný argument `join`

---

V jednoduchých ukázkách, jako jsou ty výše, stačilo tabulky a sloupečky spojit.

To prakticky není vždy ideální řešení, protože některé sloupečky můžou, ale nemusí být shodné.

In [ ]:
from pandas import DataFrame, concat

In [ ]:
df_vzorek_1 = DataFrame({"A": ["A1", "A2"], "B": ["B1", "B2"], "C": ["C1", "C2"]}, index=[1, 2])

In [ ]:
df_vzorek_2 = DataFrame({"B": ["B3", "B4"], "C": ["C3", "C4"], "D": ["D3", "D4"]}, index=[3, 4])

In [ ]:
vystup = concat([df_vzorek_1, df_vzorek_2])

In [ ]:
vystup

Pokud některá data chybějí, jsou automaticky vyplněná jako **neznámé hodnoty**.

Řešením takové situace můžeš být zavedení jiného způsobu spojování.

Tedy nepoužívat defaultní argument `join='outer'`, ale `join='inner'`:

In [ ]:
vystup_bez_na = concat([df_vzorek_1, df_vzorek_2], join='inner')

In [ ]:
vystup_bez_na

### Spojování pomocí metody `append`

---

Jde **o zastaralé řešení**, ale přesto se s ním můžeš setkat.

Jelikož je spojování natolik běžnou operací, vznikl ještě jeden způsob, který je dokonce stručnější jako `concat`.

Jde o metodu `append`:

In [ ]:
df_vzorek_3 = DataFrame({"A": ["A1", "A2"], "B": ["B1", "B2"]}, index=[1, 2])

In [ ]:
df_vzorek_4 = DataFrame({"A": ["A3", "A4"], "B": ["B3", "B4"]}, index=[3, 4])

In [ ]:
df_vzorek_3.append(df_vzorek_4)

Tato metoda přitom neupravovala původní objekty (jako `append` a `extend` pro `list`), ale vytvořila nový objekt.

### Spojování pomocí `merge`

---

Další funkcí pro spojování `DataFrame` objektů je `merge`.

Tato funkce je vhodná pro spojování DataFrame objektů, které **mají společné sloupce**.

Můžeš lépe zadávat typ spojení (parametr `how='inner' | 'outer' | 'left' | 'join'`).

Dále ti umožní definovat sloupec, nebo sloupce, na kterých chceš spojení provést (parametr `on`).

In [ ]:
from pandas import merge

In [ ]:
uzivatele_1 = {
    'jmeno': ['Alice', 'Bob', 'Charlie', 'David'],
     'vek': [25, 30, 35, 40],
     'mesto': ['Brno', 'Praha', 'Plzen', 'Ostrava']
}

In [ ]:
uzivatele_2 = {
    'jmeno': ['Alice', 'David', 'Emma', 'Frank'],
     'pocet_prijemcu': [100, 200, 150, 250]
}

In [ ]:
df_uzivatele_1 = DataFrame(uzivatele_1)

In [ ]:
df_uzivatele_2 = DataFrame(uzivatele_2)

In [ ]:
vystup = merge(df_uzivatele_1, df_uzivatele_2, on='jmeno', how='outer')

In [ ]:
vystup

Obecně platí, že pokud potřebuješ dva nebo více `DataFrame` objektů podle **společného sloupce nebo sloupců**, použij funkci `merge()`.

Pokud chceš jenom **přidat další řádky nebo sloupce** do existujícího `DataFrame` objektu, použij funkci `concat`.

In [ ]:
df_uzivatele_1

In [ ]:
df_uzivatele_2

In [ ]:
vystup_left_join = merge(df_uzivatele_1, df_uzivatele_2, on='jmeno', how='left')

In [ ]:
vystup_left_join

V ukázce výš je použitý *left join*.

Tedy ve výsledku uvidíš celou první (levou tabulku) a z druhé pouze ty záznamy, které mají ve spojovacím sloupci `jmeno` společnou hodnotu.

In [ ]:
df8 = DataFrame({
    'jmeno': ['Bob', 'Jake', 'Lisa', 'Sue'],
    'poradi': [1, 2, 3, 4]
})

In [ ]:
df9 = DataFrame({
    'jmeno': ['Bob', 'Jake', 'Lisa', 'Sue'],
    'poradi': [3, 1, 4, 2]
})

In [ ]:
vystup_konflikt = merge(df8, df9, on="jmeno")

In [ ]:
vystup_konflikt

Funkce sama doplní přípony, aby rozlišila mezi oběma původními sloupci.

Pokud potřebuješ vlastní přípony, můžeš vyzkoušet volitelný argument pro `suffixes`:

In [ ]:
vystup_vlastni_pripony = merge(df8, df9, on='jmeno', suffixes=('_levy', '_pravy'))

In [ ]:
vystup_vlastni_pripony

### Spojení pomocí indexů, `join`

---

Tato metoda slouží k propojení dvou DataFrame objektů na základě jejich indexů nebo hodnot.

Je velice podobná funkci `merge` ale je přímo součástí `DataFrame` objektu a je snazší ji aplikovat:

In [ ]:
uzivatele_1 = {
    'jmeno': ['Alice', 'Bob', 'Petr'],
     'vek': [25, 30, 35]
}

In [ ]:
uzivatele_2 = {
    'jmeno': ['Alice', 'Bob', 'Petr'],
    'pocet_prijemcu': [100, 150, 200]
}

In [ ]:
df_uzivatele_1 = DataFrame(uzivatele_1)

In [ ]:
df_uzivatele_2 = DataFrame(uzivatele_2)

In [ ]:
df_uzivatele_1 = df_uzivatele_1.set_index('jmeno')
df_uzivatele_2 = df_uzivatele_2.set_index('jmeno')

In [ ]:
vysledek_join_metody = df_uzivatele_1.join(df_uzivatele_2)

In [ ]:
vysledek_join_metody

Hlavní rozdíl mezi těmito funkcemi je způsob určení sloupce nebo sloupců, podle kterých se má propojení provést.

Metoda `join()` propojuje DataFrame objekty **na základě jejich indexů**, zatímco funkce `merge()` umožňuje propojit `DataFrame` objekty **na základě hodnoty v jednom nebo více sloupcích**.

<br>

**🧠 CVIČENÍ 🧠, procvič si spojování**

Máš dvě tabulky s informacemi o zákaznících tvé firmy.

Tabulky `zakaznici_objednavky` a `zakaznici_info`.

Tvým úkolem je propojit tyto tabulky podle zadání:
1. Použij funkci `merge()` k propojení obou tabulek podle společného klíče, kterým je sloupec `id_zakaznika`,
2. Použij funkci `concat()` k přidání sloupce `celkova_cena_objednavky`, který bude vypočítán jako součet ceny všech objednávek daného zákazníka,
3. Vyfiltruj pouze informace o zákaznících ze státu `'USA'` a ulož výslednou tabulku.

In [ ]:
import pandas as pd

In [ ]:
df_zakaznici_info = pd.DataFrame({'id_zakaznika': [1, 2, 3, 4],
                               'jmeno': ['Jan Novák', 'Petr Soukup', 'Marie Horáková', 'Jana Svobodová'],
                               'adresa': ['Hlavní 15', 'Druhá 10', 'Třetí 25', 'Čtvrtá 20'],
                               'mesto': ['Praha', 'Brno', 'Ostrava', 'New York'],
                               'stat': ['CZ', 'CZ', 'CZ', 'USA']})

In [ ]:
df_zakaznici_objednavky = pd.DataFrame({'id_zakaznika': [1, 2, 3, 4, 1, 2],
                                     'datum_objednavky': ['2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01', '2022-06-01'],
                                     'nazev_produktu': ['PC', 'Notebook', 'Monitor', 'Tiskarna', 'Myš', 'Klávesnice'],
                                     'cena': [15000, 20000, 5000, 3000, 500, 800]})

<details>
    <summary>▶️ Řešení</summary>
    
    ```python
    spojene_tabulky = pd.merge(df_zakaznici_info, df_zakaznici_objednavky, on='id_zakaznika')

    celkova_cena_objednavek = spojene_tabulky.groupby('id_zakaznika')['cena'].sum().reset_index()
    celkova_cena_objednavek = celkova_cena_objednavek.rename(columns={'cena': 'celkova_cena_objednavky'})
    spojene_tabulky = pd.concat([spojene_tabulky, celkova_cena_objednavek['celkova_cena_objednavky']], axis=1)

    spojene_tabulky = spojene_tabulky[spojene_tabulky['stat'] == 'USA']
    ```
</details>

## Agregace

Seskupování a agregace jsou procesy, které patří k základní efektivní analýze dat.

### Jednoduchá agregace

Přesto, že veškeré základní **statistické údaje** prakticky nabízí metoda `describe()`, můžeš ocenit, když stejnou statistiku můžeš aplikovat **na tebou vybrané objekty**.

Mezi nejjednodušší postupy, jak data analyzovat patří metody jako:
* `sum()`,
* `mean()`,
* `median()`,
* `min()`,
* `max()`.

Všechny tyto metody umožní získat jedno samotné číslo, které ti umožní prohlédnout podstatu zadaného datasetu.

In [2]:
from pandas import read_csv

In [4]:
df_nemovitosti = read_csv("nemovitosti.csv")

<br>

#### Odstranit nepotřebný sloupeček

In [11]:
df_nemovitosti.columns

Index(['Unnamed: 0', 'id', 'price', 'area', 'bedrooms', 'bathrooms', 'garage',
       'distance_to_center'],
      dtype='object')

In [14]:
df_bez_bezejmen = df_nemovitosti.drop("Unnamed: 0", axis=1)

In [18]:
df_bez_bezejmen.head()

,id,price,area,bedrooms,bathrooms,garage,distance_to_center
0,1,18094478,254,1,1,False,16
1,2,15315092,228,1,3,False,24
2,3,4234489,112,3,3,False,16
3,4,16586186,145,2,1,True,23
4,5,11628519,280,5,1,False,2


In [17]:
df_bez_bezejmen.loc[:, "price"].max()

19824013

In [ ]:
df_bez_bezejmen.loc[:, "price"].min()

In [ ]:
df_bez_bezejmen.loc[:, "area"].sum()

In [ ]:
df_bez_bezejmen.loc[:, "price"].mean()

<br>

Často ale není dostačující, prozkoumat data pouze jednoduchých agregačních funkcí.

Další operace, které je potřeba pochopit jsou seskupování dat podle zadaných parametrů.

### Seskupování groupby

Funkce `groupby`, původně operace z SQL jazyka, je v rámci knihovny `pandas` všestraný pomocník pro seskupování dat na základě různých kritérií.

In [2]:
from pandas import DataFrame

In [3]:
df_pokus_s_cisly = DataFrame(
    {
        'klíč': ['A', 'B', 'C', 'A', 'B', 'C'],
        'číselná hodnota': range(6)},
)

In [4]:
df_pokus_s_cisly.head(6)

,klíč,číselná hodnota
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


#### Klasické seskupení podle sloupečku

In [5]:
df_pokus_s_cisly.groupby("klíč")

Metoda standardně vrací `DataFrameGroupBy`.

Jde opět o tzv. *lazy evaluation* proces, samotný nic neprovede, pouze čeká na pokyn uživatele, který samotnou agregaci spustí.

In [6]:
df_pokus_s_cisly.groupby("klíč").sum()

,číselná hodnota
klíč,
A,3
B,5
C,7


Metoda `sum()` je pouze jednou z možností, se kterou můžeš pracovat.

### Sloupečkové označování

Stejně jako `DataFrame` můžeš označovat také *GroupBy* objekty.

In [7]:
df_pokus_s_cisly.groupby("klíč")["číselná hodnota"].median()

klíč
A    1.5
B    2.5
C    3.5
Name: číselná hodnota, dtype: float64

<br>

V uplynulé ukázce je zadaná seskupování podle sloupečku `klíč`.

Dále je vybraný pouze konkrétní sloupeček, na který chceš spustit metodu `median`.

<br>

Pokud potřebuješ nad vybraným objektem provádět některé procesy ručně, můžeš přes *GroupBy* objekt **iterovat**:

In [11]:
for (klic, hodnota) in df_pokus_s_cisly.groupby("klíč"):
    print(f"Klic: {klic}; Hodnota={hodnota.shape}")

Klic: A; Hodnota=(2, 2)
Klic: B; Hodnota=(2, 2)
Klic: C; Hodnota=(2, 2)


### Agregace

Kromě jednoduchých agregací, nabízí *GroupBy* řadu další funkcionality.

Jde o metody:
* `aggregate`,
* `filter`,
* `transform`,
* `apply`.

In [24]:
import numpy

In [25]:
rng = numpy.random.RandomState(0)

In [19]:
from pandas import DataFrame

In [26]:
df_pokus_s_cisly = DataFrame(
    {
        'klíč': ['A', 'B', 'C', 'A', 'B', 'C'],
        'data_1': range(6),
        'data_2': rng.randint(0, 10, 6)
    }
)

In [28]:
df_pokus_s_cisly.head()

,klíč,data_1,data_2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7


Statistick metody určitě nabízí spousty pomůcek.

Objekt typu *GroupBy* umí pracovat také s funkcemi, stringy a celými listy.

In [29]:
df_pokus_s_cisly.groupby("klíč").aggregate(["min", numpy.median, max])

data_1            data_2           
        min median max    min median max
klíč                                    
A         0    1.5   3      3    4.0   5
B         1    2.5   4      0    3.5   7
C         2    3.5   5      3    6.0   9

Můžeš říct, že metoda `aggregate` se používá k aplikaci **jedné nebo více agregačních funkcí na seskupená data**.

```
store,fruit,quantity_sold,price
A,apple,10,20
A,banana,15,12
B,apple,8,22
B,banana,30,10
C,apple,20,18
C,banana,25,15
```

In [30]:
ovoce_data = {'pobocka': ['A', 'A', 'B', 'B', 'C', 'C'],
        'ovoce': ['jablko', 'banan', 'jablko', 'banan', 'jablko', 'banan'],
        'prodane_mnozstvi': [10, 15, 8, 30, 20, 25],
        'cena': [20, 12, 22, 10, 18, 15]}

In [31]:
df_ovoce = DataFrame(ovoce_data)

In [35]:
df_ovoce.head(6)

,pobocka,ovoce,prodane_mnozstvi,cena
0,A,jablko,10,20
1,A,banan,15,12
2,B,jablko,8,22
3,B,banan,30,10
4,C,jablko,20,18
5,C,banan,25,15


In [36]:
vystup = df_ovoce.groupby('pobocka').aggregate({'prodane_mnozstvi': 'sum', 'cena': 'mean'})

In [37]:
vystup

,prodane_mnozstvi,cena
pobocka,,
A,25,16.0
B,38,16.0
C,45,16.5


1. Nejprve jsou hodnoty seskupení podle sloupce `pobocka`,
2. poté specifikuješ pomocí `aggregate` funkce a sloupce,
3. .. tedy sumarizovat hodnoty v `prodane_mnozstvi` a získat průměr `cena` pro každou pobočku.

### Filtrování

Filtrování ti umožní zahodit takové údaje, které nesplňují zadanou podmínku.

Metoda `filter` se používá k vybrání seskupených dat podle splnění určité podmínky.

Představme si, že máme následující dataset s informacemi o prodeji ovoce v různých obchodech:

In [41]:
def vyber_pouze_zadane_mnozstvi(x, limit: int = 35):
    return x['prodane_mnozstvi'].sum() > limit

<br>

Funkce `vyber_pouze_zadane_mnozstvi`, tedy **filtrovací funkce**, musí vracet **boolean** datový typ.

In [42]:
vysledek = df_ovoce.groupby('pobocka').filter(vyber_pouze_zadane_mnozstvi)

In [43]:
vysledek

,pobocka,ovoce,prodane_mnozstvi,cena
2,B,jablko,8,22
3,B,banan,30,10
4,C,jablko,20,18
5,C,banan,25,15


1. Nejprve seskupíš data podle sloupce `pobocka` pomocí `groupby`,
2. dále použiješ metodu `filter`, která umožňuje použít uživatelem definovanou funkci,
3. definuješ funkci, která vybere pouze pobočky s větším prodejem než je parametr `limit`,
4. metoda `filter` pak vybere pouze ty řádky, které splňují tuto podmínku.

### Transformace

Zatímco předchozí výsledky *agregace pomocí `groupby` vraceli redukované množství dat.

Transformace obvykle vrací data o stejném rozsahu jako vstupní data. Jenom upravená.

Metoda `transform` se používá k aplikaci určité **transformační funkce** na každý prvek seskupených dat.

In [56]:
def vrat_procenta_z_celkoveho_prodeje(udaj):
    return round(udaj / udaj.sum() * 100, 1)

<br>

Vytvoření nového sloupečku `procento_z_celkoveho_prodeje`:

In [57]:
df_ovoce['procento_z_celkoveho_prodeje'] = df_ovoce.groupby('pobocka')['prodane_mnozstvi'] \
    .transform(vrat_procenta_z_celkoveho_prodeje)

In [58]:
df_ovoce.head(6)

,pobocka,ovoce,prodane_mnozstvi,cena,procento_z_celkoveho_prodeje
0,A,jablko,10,20,40.0
1,A,banan,15,12,60.0
2,B,jablko,8,22,21.1
3,B,banan,30,10,78.9
4,C,jablko,20,18,44.4
5,C,banan,25,15,55.6


1. Nejprve seskupíš data podle sloupce `pobocka` pomocí `groupby`,
2. dále použiješ metodu `transform`, která umožňuje použít uživatelem definovanou funkci,
3. definuješ funkci, která vybere vypočítá procentuální vyjádření prodaného zboží pro pobočku,
4. přidáš nový sloupeček `procento_z_celkoveho_prodeje`.

### Metoda apply

Metoda `apply` ti dovol9 se používá k aplikaci uživatelem definované funkce na seskupená data.

In [ ]:
def total_revenue(group):
    group['revenue'] = group['quantity_sold'] * group['price']
    return group

# Použití groupby s metodou apply
result = df.groupby('store').apply(total_revenue)

print(result)

---